# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [7]:
import nltk
import numpy as np

from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from collections import Counter
from nltk.tokenize import sent_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eyer8\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
dvach = open('2ch_corpus.txt', encoding='utf-8').read()
news = open('lenta.txt', encoding='utf-8').read()

In [9]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [10]:
#применяем нормализацию к текстам
norm_dvach = normalize(dvach)
norm_news = normalize(news)
#cоздаем частотные словари
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [11]:
vocab_dvach.most_common(10)

[('и', 55892),
 ('в', 48853),
 ('не', 46602),
 ('на', 29660),
 ('что', 26668),
 ('я', 21734),
 ('а', 21310),
 ('с', 21080),
 ('это', 17727),
 ('ты', 15469)]

In [14]:
vocab_news.most_common(10)

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

In [13]:
#переводим абсолютные частоты в вероятности, разделив количество употреблений слова на общее число слов в корпусе
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(10)

[('и', 0.030066580918921042),
 ('в', 0.02628001641794979),
 ('не', 0.02506911192985684),
 ('на', 0.015955320798239428),
 ('что', 0.014345802260534357),
 ('я', 0.011691602907246653),
 ('а', 0.011463516055646737),
 ('с', 0.011339789697467536),
 ('это', 0.009536074571489897),
 ('ты', 0.008321404498582796)]

In [15]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(10)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205)]

In [17]:
sentences_dvach = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [18]:
# берем отрывок из 50 предложений для расчета перплексии
test_dvach = sentences_dvach [-50:]
test_news = sentences_news[-50:]

# убираем эти предложения из основной выборки
sentences_dvach = sentences_dvach[:-50]
sentences_news = sentences_news[:-50]


In [21]:
# ищем кол-во вхождений каждой n-граммы и проходимся по токенам
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [24]:
#Создаем частотные словари униграм, биграм и триграм
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence, n=2))
    trigrams_dvach.update(ngrammer(sentence, n=3))
    
    
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, n=2))
    trigrams_news.update(ngrammer(sentence, n=3))

In [25]:
trigrams_news.most_common(10)

[('<start> об этом', 1578),
 ('<start> по словам', 1549),
 ('сообщает риа новости', 1324),
 ('со ссылкой на', 1242),
 ('риа новости <end>', 1228),
 ('<start> кроме того', 1070),
 ('<start> как сообщает', 1064),
 ('<start> напомним что', 1005),
 ('по его словам', 899),
 ('<start> по его', 868)]

In [26]:
trigrams_dvach.most_common(10)

[('<start> у меня', 923),
 ('<start> я не', 559),
 ('<start> потому что', 551),
 ('<start> ну и', 524),
 ('на самом деле', 501),
 ('в том что', 484),
 ('<start> да и', 469),
 ('<start> так что', 415),
 ('<start> а вот', 371),
 ('до сих пор', 357)]

### Генерация текста


In [28]:
#создаем матрицу вероятностей для Двача

matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}


for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2 
    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word3]] =  (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[bigram])

MemoryError: Unable to allocate 1.42 TiB for an array with shape (1032067, 189484) and data type float64

In [29]:
#создаем матрицу вероятностей для новостей
matrix_news = np.zeros((len(bigrams_news), 
                   len(unigrams_news)))
id2unigram_news = list(unigrams_news)
word2id_news_uni = {word:i for i, word in enumerate(id2unigram_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id_news[bigram], word2id_news_uni[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

MemoryError: Unable to allocate 666. GiB for an array with shape (769185, 116254) and data type float64

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

2. Что такое сглаживание (smoothing)?